In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import skimage
import pylab as plt
from importlib import reload
from datetime import datetime

sys.path.insert(0, '../src')
import deepleeo.dataset.data_augment as dtaug
import deepleeo.dataset.utils as dsutils
import deepleeo.utils.geofunctions as gf
from deepleeo.networks import fcn

reload(dtaug)
reload(dsutils)
reload(fcn)
reload(gf)

/home/raian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'deepleeo.utils.geofunctions' from '../src/deepleeo/utils/geofunctions.py'>

# Load input Dataset

In [2]:
DATA_DIR = os.path.join(os.path.abspath(os.path.dirname("__file__")), '../', 'data_real', 'generated')
DATASET_FILE = os.path.join(DATA_DIR, 'samples_dataset.npz')
model_dir = os.path.join(DATA_DIR, 'tf_logs', "test_%s" % datetime.now().strftime('%d_%m_%Y-%H_%M_%S'))

In [3]:
raster_path = os.path.join(DATA_DIR, "..", "Landsat8_225-64_17-07-2016-R6G5B4.tif")

In [4]:
dataset = np.load(DATASET_FILE)

print("Data Loaded:")
print("  -> Images: ", len(dataset["images"]))
print("  -> Labels: ", len(dataset["labels"]))
print("  -> Classes: ", len(dataset["classes"]))

print("Images shape: ", dataset["images"][0].shape)
print("Labels shape: ", dataset["labels"][0].shape)

Data Loaded:
  -> Images:  5000
  -> Labels:  5000
  -> Classes:  4
Images shape:  (128, 128, 3)
Labels shape:  (128, 128, 1)


In [5]:
#plt.figure(figsize=(4,4))
#img_plt = skimage.img_as_float(dataset["images"][0])
#img_plt = dataset["images"][0]
#plt.imshow(img_plt)
#plt.axis('off')

# Perform Data Augmentation

In [6]:
angles = [90, 180, 270]
rotated_imgs = dtaug.rotate_images(dataset["images"], angles)
flipped_imgs = dtaug.flip_images(dataset["images"])

new_dataset = {}
new_dataset["images"] = np.concatenate((dataset["images"], rotated_imgs))
new_dataset["images"] = np.concatenate((new_dataset["images"], flipped_imgs))

rotated_lbls = dtaug.rotate_images(dataset["labels"], angles)
flipped_lbls = dtaug.flip_images(dataset["labels"])

new_dataset["labels"] = np.concatenate((dataset["labels"], rotated_lbls))
new_dataset["labels"] = np.concatenate((new_dataset["labels"], flipped_lbls))

new_dataset["classes"] = dataset["classes"]

print("Data Augmentation Applied:")
print("  -> Images: ", new_dataset["images"].shape)
print("  -> Labels: ", new_dataset["labels"].shape)

Data Augmentation Applied:
  -> Images:  (35000, 128, 128, 3)
  -> Labels:  (35000, 128, 128, 1)


# Split dataset between train, test and validation data

In [7]:
train_images, test_images, valid_images, train_labels, test_labels, valid_labels = dsutils.split_dataset(new_dataset)

print("Splitted dataset:")
print("  -> Train images: ", train_images.shape)
print("  -> Test images: ", test_images.shape)
print("  -> Validation images: ", valid_images.shape)
print("  -> Train Labels: ", train_labels.shape)
print("  -> Test Labels: ", test_labels.shape)
print("  -> Validation Labels: ", valid_labels.shape)

Splitted dataset:
  -> Train images:  (24500, 128, 128, 3)
  -> Test images:  (10500, 128, 128, 3)
  -> Validation images:  (0, 128, 128, 3)
  -> Train Labels:  (24500, 128, 128, 1)
  -> Test Labels:  (10500, 128, 128, 1)
  -> Validation Labels:  (0, 128, 128, 1)


# Train the Network

In [8]:
params = {
    "epochs": 300,
    "batch_size": 400,
    "learning_rate": 0.01,
    "class_names": new_dataset["classes"]
}

In [ ]:
reload(fcn)

fcn.fcn_train(train_images, test_images, train_labels, test_labels, params, model_dir)

INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'].  Variables are going to be placed on ['/CPU:0'].  Consolidation device is going to be /CPU:0.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_service': None, '_model_dir': '/home/raian/doutorado/DeepLeEO/notebooks/../data_real/generated/tf_logs/test_22_08_2018_19_47_03', '_device_fn': None, '_save_summary_steps': 100, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_task_id': 0, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f55400f9908>, '_num_ps_replicas': 0, '_master': '', '_save_checkpoints_secs': 600, '_task_type': 'worker', '_train_distribute': None, '_log_step_count_steps': 100, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_is_chief': True}
Epoch  1
---------------
T

In [10]:
def generate_sequential_chips(img_array, chip_size=128):
    x_size, y_size, nbands = img_array.shape
    
    chips = []
    for y_start in range(0, y_size, chip_size):
        y_end = y_start + chip_size
        
        if y_end > y_start:
            y_end = y_size
            y_start = y_end - chip_size
        
        for x_start in range(0, x_size, chip_size):
            x_end = x_start + chip_size
            
            if x_end > x_size:
                x_end = x_size
                x_start = x_end - chip_size
                
            chip_array = img_array[x_start:x_end, y_start:y_end, :]
            
            chips.append(chip_array)
        
    return chips

reload(gf)
raster_array = gf.load_image(raster_path, 0)
chips = generate_sequential_chips(raster_array)

print(len(chips))

3660


In [13]:
reload(fcn)

trained_model = "/home/raian/doutorado/DeepLeEO/data_real/generated/tf_logs/test_22_08_2018-20_24_36"

predictions = fcn.fcn_predict(chips, params=params, model_dir=trained_model)


Classifying image with structure  (3660, 128, 128, 3) ...
